# Importing Libraries

In [13]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,GRU,Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow import keras
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# Path and Pre-Processing

In [15]:
DATA_PATH_train = os.path.join(r'E:\M.Tech\Dissertation Stage 2 - Final\Experimental\Final Character\Alpha_train')
DATA_PATH_test = os.path.join(r'E:\M.Tech\Dissertation Stage 2 - Final\Experimental\Final Character\Alpha_test')
#no_sequences = 50 #number of videos
sequence_length = 1000     #number of frames per video

In [16]:
ax_train = os.listdir(r'E:\M.Tech\Dissertation Stage 2 - Final\Experimental\Final Character\Alpha_train')
actions_train = []
label_map_train = {}
ax_test = os.listdir(r'E:\M.Tech\Dissertation Stage 2 - Final\Experimental\Final Character\Alpha_test')
actions_test = []
label_map_test = {}

In [17]:
for a in range(len(ax_train)):
    label_map_train[ax_train[a]]=a
    actions_train.append(ax_train[a])
for a in range(len(ax_test)):
    label_map_test[ax_test[a]]=a
    actions_test.append(ax_test[a])  
    
print(label_map_train,label_map_test)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'Reset': 18, 'S': 19, 'Space': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27} {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'Reset': 18, 'S': 19, 'Space': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27}


In [18]:
sequences_train, labels_train = [],[]
for axs_train in actions_train:
    for files in os.listdir(r'Alpha_train\\'+axs_train):
        res = np.load(r'Alpha_train\\'+axs_train+'\\'+files)
        sequences_train.append(res)
        labels_train.append(label_map_train[axs_train])
        #sequences_train.append(res)
        #labels_train.append(label_map_train[axs_train])
        
sequences_test, labels_test = [],[]
for axs_test in actions_test:
    for files in os.listdir(r'E:\M.Tech\Dissertation Stage 2 - Final\Experimental\Final Character\Alpha_test\\'+axs_test):
        res = np.load(r'E:\M.Tech\Dissertation Stage 2 - Final\Experimental\Final Character\Alpha_test\\'+axs_test+'\\'+files)
        sequences_test.append(res)
        labels_test.append(label_map_test[axs_test])
        #sequences_train.append(res)
        #labels_train.append(label_map_train[axs_train])
        

In [19]:
print(np.array(sequences_train).shape)
print(np.array(sequences_test).shape)
print(np.array(labels_train).shape)
print(np.array(labels_test).shape)

(22400, 126)
(5600, 126)
(22400,)
(5600,)


In [20]:
train_x = np.array(sequences_train)
test_x = np.array(sequences_test) 
train_y = to_categorical(labels_train).astype(int)
test_y = to_categorical(labels_test).astype(int)

In [42]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(22400, 126)
(22400, 28)
(5600, 126)
(5600, 28)


In [43]:
log_dir = os.path.join('Dense_Neural_Network')
tb_callback = TensorBoard(log_dir = log_dir)

In [44]:
from keras.layers import LeakyReLU

# define the keras model
mod = Sequential()
mod.add(Dense(128, input_shape=(126,), activation='relu'))
mod.add(Dropout(0.3))
mod.add(Dense(128, activation = 'relu'))
mod.add(Dropout(0.3))
mod.add(Dense(64, activation = 'relu'))
mod.add(Dropout(0.3))
mod.add(Dense(32, activation = 'relu'))
mod.add(Dropout(0.3))
mod.add(Dense(28, activation='sigmoid'))

In [45]:
mod.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
mod.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               16256     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)               

In [46]:
mod.fit(train_x, train_y, epochs = 20, batch_size=126,callbacks = [tb_callback] )
#scores = model.evaluate(test_x, test_y)

Epoch 1/20
178/178 [==============================] - 4s 17ms/step - loss: 2.7017 - accuracy: 0.1858
Epoch 2/20
178/178 [==============================] - 1s 8ms/step - loss: 1.4639 - accuracy: 0.5068
Epoch 3/20
178/178 [==============================] - 1s 8ms/step - loss: 0.9538 - accuracy: 0.6672
Epoch 4/20
178/178 [==============================] - 1s 8ms/step - loss: 0.7343 - accuracy: 0.7497
Epoch 5/20
178/178 [==============================] - 1s 8ms/step - loss: 0.6170 - accuracy: 0.7900
Epoch 6/20
178/178 [==============================] - 2s 9ms/step - loss: 0.5245 - accuracy: 0.8300
Epoch 7/20
178/178 [==============================] - 1s 8ms/step - loss: 0.4534 - accuracy: 0.8568
Epoch 8/20
178/178 [==============================] - 2s 9ms/step - loss: 0.3936 - accuracy: 0.8796
Epoch 9/20
178/178 [==============================] - 2s 9ms/step - loss: 0.3590 - accuracy: 0.8936
Epoch 10/20
178/178 [==============================] - 2s 9ms/step - loss: 0.3203 - accuracy: 0.908

In [47]:
np.argmax(model.predict(test_x[2].reshape(-1,126)))

0

In [48]:
tt = {0:'A',1:'B',2:'C',3:'D',4:'E',
     5:'F',6:'G',7:'H',8:'I',9:'J',
     10:'K',11:'L',12:'M',13:'N',14:'O',
     15:'P',16:'Q',17:'R',18:'Reset',19:'S',
     20:'Space',21:'T',22:'U',23:'V',24:'W',
     25:'X',26:'Y',27:'Z'}

def pred(test_path):
    test_path = np.load(test_path)
    x = model.predict(test_path.reshape(-1,126))
    return tt[np.argmax(x)]
    #op = floor(x[0])

In [49]:
pred(r'E:\M.Tech\Dissertation Stage 2 - Final\Final Character\Alpha_test\E\820.npy')

'E'

In [50]:
model.save('Perfect_28_Character_relu.h5')

# Test


In [36]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from preprocess import mediapipe_detection,landmarks

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [37]:
def draw_styled_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (121,22,76), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (121,44,250), thickness = 2, circle_radius = 2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (245,117,66), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (245,66,230), thickness = 2, circle_radius = 2))

def extract_keypoints(results):
    left_hand = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark] ).flatten() if results.left_hand_landmarks else np.zeros(63)
    right_hand = np.array(([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark] )).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([left_hand,right_hand])

In [38]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.4) as holistic:
    
    while True: 
        
        _,frame = cap.read()
        image,results = mediapipe_detection(frame,holistic)
        draw_styled_landmarks(image,results)
        

        #cv2.imshow("sign_videos",image)

        keypoints = extract_keypoints(results)
        max_frame = []
        pp = pred(keypoints)
        cv2.putText(image,'Image for{}'.format(pp),(15,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,255),1,cv2.LINE_AA)
        cv2.imshow("sign_videos",image)
        '''
        for xx in range(0,20):
            pp = max_frame.append(pred(keypoints))
        else:
            print(max(pp))

        continue

        '''
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()